# imports

In [3]:
from dash import Dash, dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template
import plotly.express as px
import pandas
import numpy as np
import random
from numpy.random import default_rng
import datetime

# dash app

In [8]:

df_edu = pandas.read_csv('data/states_all.csv').iloc[:, 1:]
print(df_edu.head())

app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE])

df_edu_chain = (
    df_edu
    .assign(
        Avg_Math=lambda df: df[['AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE']].mean(axis=1),
        Avg_Reading=lambda df: df[['AVG_READING_4_SCORE', 'AVG_READING_8_SCORE']].mean(axis=1),
    )
)

all_options = {
    "By Subject": [
        {"label": "Math", "value": "Avg_Math"},
        {"label": "Reading", "value": "Avg_Reading"}
    ],
    "By Grade and Subject": [
        {"label": "4th Grade Math", "value": "AVG_MATH_4_SCORE"},
        {"label": "8th Grade Math", "value": "AVG_MATH_8_SCORE"},
        {"label": "4th Grade Reading", "value": "AVG_READING_4_SCORE"},
        {"label": "8th Grade Reading", "value": "AVG_READING_8_SCORE"},
    ],
}

app.layout = html.Div([
    dcc.RadioItems(
        id='report-type-radio',
        options=["By Subject", "By Grade and Subject"],
        value="By Subject",
    ),
    html.Hr(),
    dcc.RadioItems(id="metric-radio"),
    html.Br(),
    dcc.Graph(id="metric-bar"),
])


@app.callback(
    Output(component_id="metric-radio", component_property="options"),
    Input(component_id="report-type-radio", component_property="value")
)
def set_metrics_options(selected_report: str):
    return all_options[selected_report]


@app.callback(
    Output(component_id="metric-bar", component_property="figure"),
    Input(component_id="metric-radio", component_property="value")
)
def plot_bar(metric: str):
    if metric is None:
        raise PreventUpdate

    figure = px.bar(
        (
            df_edu_chain
            .groupby("STATE", as_index=False)
            .agg({metric: "mean"})
            .sort_values(metric, ascending=False)
        ),
        x="STATE",
        y=metric,
    )
    return figure


if __name__ == '__main__':
    # host="127.0.0.1"
    app.run_server(debug=True, port=8052)

# eof

        STATE  YEAR  ENROLL  TOTAL_REVENUE  FEDERAL_REVENUE  STATE_REVENUE  \
0     ALABAMA  1992     NaN      2678885.0         304177.0      1659028.0   
1      ALASKA  1992     NaN      1049591.0         106780.0       720711.0   
2     ARIZONA  1992     NaN      3258079.0         297888.0      1369815.0   
3    ARKANSAS  1992     NaN      1711959.0         178571.0       958785.0   
4  CALIFORNIA  1992     NaN     26260025.0        2072470.0     16546514.0   

   LOCAL_REVENUE  TOTAL_EXPENDITURE  INSTRUCTION_EXPENDITURE  \
0       715680.0          2653798.0                1481703.0   
1       222100.0           972488.0                 498362.0   
2      1590376.0          3401580.0                1435908.0   
3       574603.0          1743022.0                 964323.0   
4      7641041.0         27138832.0               14358922.0   

   SUPPORT_SERVICES_EXPENDITURE  ...  GRADES_4_G  GRADES_8_G  GRADES_12_G  \
0                      735036.0  ...     57948.0     58025.0      411